In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, StopWordsRemover, HashingTF, IDF, ChiSqSelector
from pyspark.ml.feature import Tokenizer, Normalizer, StringIndexer
from pyspark.sql.functions import col, lower

from pyspark.ml import Pipeline
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from pyspark.mllib.util import MLUtils

In [ ]:
partial_dataset = "/user/dic24_shared/amazon-reviews/full/reviews_devset.json"
full_dataset = "/user/dic24_shared/amazon-reviews/full/reviewscombined.json"


In [ ]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("TFIDF_ChiSq_Selection") \
    .getOrCreate()

try:
    df = spark.read.json(partial_dataset)
    print("File read successfully.")
    
    # Show the schema and some data
    df.printSchema()
    df.show(5)
    
except Exception as e:
    print(f"Error: {e}")

## Part 1: RDDs


## Part 2: Datasets/DataFrames: Spark ML and Pipelines

In [ ]:
# casefolding
df = df.withColumn("reviewText", lower(col("reviewText")))

In [ ]:
# convert category to numeric
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")

#Creating a pipeline:

# 1. tokenize
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern=r'\s+|\t+|\d+|[(){}.!?,;:+=-_"\`~#@&*%€$§\\/]+', gaps=True)

# 3. stopwords removal
stopwords_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered_words")


# 4. TF-IDF calculation with HashingTF
hashingTF = HashingTF(inputCol=stopwords_remover.getOutputCol(), outputCol="rawFeatures", numFeatures=20)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Chi-square
selector = ChiSqSelector(numTopFeatures=2000, featuresCol=hashingTF.getOutputCol(),
                         outputCol="selectedFeatures", labelCol=indexer.getOutputCol())

pipeline = Pipeline(stages=[tokenizer, stopwords_remover, hashingTF, idf, selector])

# Training the pipeline
model = pipeline.fit(df)
result = model.transform(df)

# Writing the result to a file
terms = result.select("selectedFeatures").collect()
with open("output_ds.txt", "w") as f:
    for term in terms:
        f.write(f"{term}\n")

# Stopping Spark
spark.stop()

## Part 3: Text Classification

In [ ]:
# Initializing Spark
spark = SparkSession.builder.appName("Part 3 Classification").getOrCreate()

# Loading data
data = spark.read.json("/path/to/your/data.json")

# Creating the pipeline (adding SVM)
normalizer = Normalizer(inputCol="selectedFeatures", outputCol="normFeatures")
svm = LinearSVC(featuresCol="normFeatures", labelCol="category")
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, selector, normalizer, svm])

# Splitting the data
# split in train, test, validation
train_df, test_df, validation_df = df.randomSplit([0.7, 0.15, 0.15], seed=42)


# Show the count of each set
print("Training set count:", train_df.count())
print("Test set count:", test_df.count())
print("Validation set count:", validation_df.count())

In [ ]:
# ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(selector.numTopFeatures, [2000, 500])  # number of features selected
             .addGrid(svm.regParam, [0.01, 0.1, 1.0])  # Regularization parameter
             .addGrid(svm.maxIter, [10, 50])  # Maximum number of iterations
             .build())

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol=indexer.getOutputCol(), metricName="f1"),
                          numFolds=3)

# Fit the pipeline to training texts.
model = crossval.fit(train_df)

# Evaluating the model
predictions = model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(labelCol="category", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)

print(f"F1 Score: {f1_score}")

# Stopping Spark
spark.stop()